In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [3]:
from sklearn.metrics import accuracy_score

In [4]:
data_a = pd.read_csv('arrh.csv')
data_a.head()

,age,sex,height,weight,QRSduration,PRinterval,Q-Tinterval,Tinterval,Pinterval,QRS,...,chV6_QwaveAmp,chV6_RwaveAmp,chV6_SwaveAmp,chV6_RPwaveAmp,chV6_SPwaveAmp,chV6_PwaveAmp,chV6_TwaveAmp,chV6_QRSA,chV6_QRSTA,class
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,0
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,0
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,0
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,0


In [5]:
x = data_a.iloc[:,:-1]
y = data_a.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=1, stratify=y)


In [6]:

grid_search = GridSearchCV(rf(random_state=1), param_grid=
                          {'max_depth':[2,5,10]})
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=1),
             param_grid={'max_depth': [2, 5, 10]})

In [7]:
depth = grid_search.best_params_['max_depth']
depth

10

In [10]:
var_trans = SelectKBest(f_classif, k=3).fit(x_train, y_train)
classifier = rf(random_state=1, max_depth=depth).fit(var_trans.transform(x_train), y_train)
accuracy_score(classifier.predict(var_trans.transform(x_test)), y_test)

C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 19  67  69  83 131 132 139 141 143 145 151 156 157 164 204 264 274] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


0.5714285714285714

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

In [12]:
pipe = Pipeline([
    ('feature_selection', SelectKBest(f_classif)),
    ('classifier', rf(random_state=1))
])

params = dict(
feature_selection__k=[2,3,4],
classifier__max_depth=[5,10,20]
)

auc_scorer = make_scorer(roc_auc_score)
accuracy_scorer = make_scorer(accuracy_score)

# the configuration with the highest scores is chosen
# If I use a custom scoring function to distinguish fp, fn same thing applies
grid_search = GridSearchCV(pipe, param_grid=params, cv=3, scoring=accuracy_scorer)
gs_obj = grid_search.fit(x_train, y_train)
gs_obj.best_params_

C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 19  21  35  37  38  62  67  69  71  83  85  86 103 105 115 119 127 131
 132 139 141 143 144 145 151 153 156 157 164 204 234 244 254 264 274] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 19  25  36  45  67  69  72  83  84 118 131 132 139 141 143 145 151 154
 156 157 164 204 264 274] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\anaconda3\lib\site-packag

C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 19  21  35  37  38  62  67  69  71  83  85  86 103 105 115 119 127 131
 132 139 141 143 144 145 151 153 156 157 164 204 234 244 254 264 274] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 19  25  36  45  67  69  72  83  84 118 131 132 139 141 143 145 151 154
 156 157 164 204 264 274] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
C:\anaconda3\lib\site-packag

{'classifier__max_depth': 5, 'feature_selection__k': 3}

In [13]:
import pickle

In [14]:
#file stream process doesn't block other processes if an exception is raised
with open('pipe.pkl', 'wb') as file:
    pickle.dump(gs_obj, file)

In [15]:
with open('pipe.pkl', 'rb') as file:
    gs_obj = pickle.load(file)
    
print(accuracy_score(gs_obj.predict(x_test), y_test))

0.5604395604395604


In [55]:
from sklearn.preprocessing import FunctionTransformer

def negate_second_column(X, m=1):
    Z = X.values.copy()
    Z[:,m] = -Z[:,m]
    return Z

In [83]:
pipe = Pipeline([
    ('ft', FunctionTransformer(negate_second_column)),
    ('classifier', rf(random_state=1))
])
    
params = dict(
#ft__m=[1,2,3],
classifier__max_depth=[1,2,3]
)

auc_scorer = make_scorer(roc_auc_score)

grid_search = GridSearchCV(pipe, param_grid=params, cv=3)
grid_search.fit(x_train, y_train).best_params_

{'classifier__max_depth': 3}